In [1]:
### This code is based on the following code bases:
### https://machinelearningmastery.com/how-to-train-a-progressive-growing-gan-in-keras-for-synthesizing-faces/
### And has borrowed some functionalities from:
### https://github.com/MSC-BUAA/Keras-progressive_growing_of_gans/blob/master/Progressive%20growing%20of%20GANs/train.py
### https://github.com/PacktPublishing/Hands-On-Generative-Adversarial-Networks-with-Keras/blob/ac1b8af1678af352e7e9efdcc6a3e829c6aed294/Chapter09/train_wgan_gp.py
### and has been further developed by Monika Lezanska and Bart Driessen for our thesis on generating neuroimaging data.
### The dataset used in this study can be downloaded from: https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri

import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import math
from numpy.random import randn, randint
from skimage.transform import resize, rotate
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, UpSampling2D, AveragePooling2D, LeakyReLU, Layer, Add
import keras
from keras.initializers import RandomNormal
from keras import backend
from functools import partial
import pandas as pd

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

os.chdir('/content/gdrive/MyDrive/master_thesis/Notebooks/pggan')

from custom_layers import *
from losses import *

from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"wasserstein_loss": wasserstein_loss})

from model import *
from preprocessing import *
from helper import *


Mounted at /content/gdrive


In [2]:
def train_epochs(g_model, d_model, gan_model, dataset, n_epochs, n_batch, fadein=False, output_folder="results4"):
  """
  Function to train a single model, either tuned or faded, on just one resolution
  Parameters:
    g_model: the generator 
    d_model: the discriminator
    gan_model: the combined model to train the generator
    dataset: the dataset, preprocessed / rescaled
    n_epochs: the number of epochs
    n_batch: the batch size
    fadein: boolean checking whether it's the faded model that's being trained
    output_folder: output folder to store the losses
  Returns:
    nothing but trains both discriminator and generator, and stores the losses
  """
  bat_per_epo = int(dataset.shape[0] / n_batch)
  n_steps = bat_per_epo * n_epochs
  half_batch = int(n_batch / 2)
  losses = []
  for i in range(n_steps):
    if fadein:
      update_fadein([g_model, d_model, gan_model], i, n_steps)
    X_real, y_real = generate_real_samples(dataset, half_batch)
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    d_loss1 = d_model.train_on_batch(X_real, y_real)
    d_loss2 = d_model.train_on_batch(X_fake, y_fake)
    z_input = generate_latent_points(latent_dim, n_batch)
    y_real2 = np.ones((n_batch, 1))
    g_loss = gan_model.train_on_batch(z_input, y_real2)
    print('>%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))
    losses.append({'step':i+1, 'd_loss1':d_loss1, 'd_loss2':d_loss2, 'g_loss':g_loss})
  faded = "faded" if fadein else "tuned"
  pd.DataFrame(losses).to_csv(output_folder+"/"+str(dataset.shape[-2])+"x"+str(dataset.shape[-2])+faded+"losses.csv")

def train(g_models, d_models, gan_models, dataset, latent_dim, e_norm, e_fadein, n_batch, pretrained_mods=None, faded=False, output_folder="results4"):
  """
  train the generators and the discriminators of the entire model - multiple growing stages, both tuned and faded
  Parameters:
    g_models: generator models
    d_models: discriminator models
    gan_models: combined models to train generator
    dataset: the input dataset (unpreprocessed)
    latent_dim: the size of the input noise vector
    e_norm: nr epochs for the tuned model
    e_fadein: nr epochs for the faded model
    n_batch: batch size
    pretrained_mods: list of three models that have already been trained up to a certain resolution. in form [g_model, d_model, gan_model]
    faded: boolean indicating whether last one of the pretrained models was faded
    output_folder: output folder to store results
  Returns:
    no output but does the training and writes all results via the summarize_performance and train_epochs methods
  """
  if not pretrained_mods:
    g_normal, d_normal, gan_normal = g_models[0][0], d_models[0][0], gan_models[0][0]
    gen_shape = g_normal.output_shape
    scaled_data = scale_dataset(dataset, gen_shape[1:])
    print('Scaled Data', scaled_data.shape)
    train_epochs(g_normal, d_normal, gan_normal, scaled_data, e_norm[0], n_batch[0], output_folder=output_folder)
    summarize_performance('tuned', g_normal, d_normal, gan_normal, latent_dim, output_folder=output_folder)
    starting_i = 1
  else:
    a_mod = pretrained_mods[0]
    gen_shape = a_mod.output_shape
    scaled_data = scale_dataset(dataset, gen_shape[1:])
    starting_i = int(math.log(a_mod.output_shape[-2])/math.log(2))-1
    if faded:
      starting_i -= 1
    print('Scaled Data', scaled_data.shape)
  for i in range(starting_i, len(g_models)):
    print("batchsize",n_batch[i])
    print("epochs",e_norm[i])
    [g_normal, g_fadein] = g_models[i]
    [d_normal, d_fadein] = d_models[i]
    [gan_normal, gan_fadein] = gan_models[i]
    gen_shape = g_normal.output_shape
    scaled_data = scale_dataset(dataset, gen_shape[1:])
    print('Scaled Data', scaled_data.shape)
    if not faded or i > starting_i:
      train_epochs(g_fadein, d_fadein, gan_fadein, scaled_data, e_fadein[i], n_batch[i], True, output_folder=output_folder)
      summarize_performance('faded', g_fadein, d_fadein, gan_fadein, latent_dim, output_folder=output_folder)
    train_epochs(g_normal, d_normal, gan_normal, scaled_data, e_norm[i], n_batch[i], output_folder=output_folder)
    summarize_performance('tuned', g_normal, d_normal, gan_normal, latent_dim, output_folder=output_folder)


In [3]:
dataset = load_real_samples('img_align_braint_128.npz')
print('Loaded', dataset.shape)

Loaded (641, 128, 128, 3)


In [4]:
n_blocks = 6 ### number of growth phases, e.g. 6 == [4, 8, 16, 32, 64, 128]
latent_dim = 100 ### size of the latent space
d_models = define_discriminator(n_blocks)
g_models = define_generator(latent_dim, n_blocks)
gan_models = define_composite(d_models, g_models)

n_batch = [32, 32, 32, 32, 32, 32]
n_epochs = [100, 100, 100, 100, 200, 300]

input_folder = "final_results"
model_size = "064x064"
model_type = "tuned" ### was the last trained model "tuned" of "faded"?
g_model = load_model(input_folder+'/g_model_'+model_size+"-"+model_type)
d_model = load_model(input_folder+'/d_model_'+model_size+"-"+model_type)
gan_model = load_model(input_folder+'/gan_model_'+model_size+"-"+model_type)
train(g_models, d_models, gan_models, dataset, latent_dim, n_epochs, n_epochs, n_batch, [g_model, d_model, gan_model], faded=model_type=="faded", output_folder="final_results")

Scaled Data (1923, 64, 64, 3)
batchsize 32
epochs 300
Scaled Data (1923, 128, 128, 3)
>1, d1=0.000, d2=0.000 g=-0.000
>2, d1=-0.000, d2=0.000 g=-0.000
>3, d1=-0.000, d2=0.000 g=-0.000
>4, d1=-0.000, d2=0.000 g=-0.000
>5, d1=-0.000, d2=0.000 g=-0.000
>6, d1=-0.000, d2=0.000 g=-0.000
>7, d1=-0.000, d2=0.000 g=-0.000
>8, d1=-0.000, d2=0.000 g=0.000
>9, d1=-0.000, d2=0.000 g=0.000
>10, d1=-0.000, d2=0.000 g=0.000
>11, d1=-0.000, d2=0.000 g=0.000
>12, d1=-0.000, d2=0.000 g=0.000
>13, d1=-0.000, d2=0.000 g=-0.000
>14, d1=-0.000, d2=0.000 g=-0.000
>15, d1=-0.000, d2=0.000 g=-0.000
>16, d1=-0.000, d2=0.000 g=-0.000
>17, d1=-0.000, d2=0.000 g=-0.000
>18, d1=-0.000, d2=0.000 g=-0.000
>19, d1=-0.000, d2=0.000 g=-0.000
>20, d1=-0.000, d2=0.000 g=-0.000
>21, d1=-0.000, d2=0.000 g=-0.000
>22, d1=-0.000, d2=0.000 g=-0.000
>23, d1=-0.000, d2=0.000 g=-0.000
>24, d1=-0.000, d2=0.000 g=-0.000
>25, d1=-0.000, d2=0.000 g=-0.000
>26, d1=-0.000, d2=0.000 g=-0.000
>27, d1=-0.000, d2=0.000 g=-0.000
>28, d1=-0.

KeyboardInterrupt: ignored